<a href="https://colab.research.google.com/github/Oelebrashy/Machine-Learning/blob/main/Housing_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
!pip install category_encoders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

# Load the datasets
train_path = '/content/drive/My Drive/House Prices-Advanced Regression Techniques/train.csv'
test_path = '/content/drive/My Drive/House Prices-Advanced Regression Techniques/test.csv'
data_description_path = '/content/drive/My Drive/House Prices-Advanced Regression Techniques/data_description.txt'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Preserve the target variable 'SalePrice' and the 'Id' column before encoding
train_id = train['Id']
train_target = train['SalePrice']
test_id = test['Id']

# Display the first few rows of the training dataset
print("Training Data:")
print(train.head())

print("\nTest Data:")
print(test.head())

# Check for missing values
print("\nMissing values in training set:")
print(train.isnull().sum()[train.isnull().sum() > 0])

print("\nMissing values in test set:")
print(test.isnull().sum()[test.isnull().sum() > 0])

# Data description
with open(data_description_path, 'r') as file:
    data_description = file.read()

print("\nData Description:")
print(data_description)

# Initial data visualization
plt.figure(figsize=(10, 6))
sns.histplot(train['SalePrice'], kde=True)
plt.title('Distribution of Sale Prices')
plt.xlabel('SalePrice')
plt.ylabel('Frequency')
plt.show()

# Display the data types of each column
print("\nData Types in Training Set:")
print(train.dtypes)

In [ ]:
# Fill missing values for categorical features
cat_fill_values = {
    'Alley': 'None', 'MasVnrType': 'None', 'BsmtQual': 'NA', 'BsmtCond': 'NA',
    'BsmtExposure': 'NA', 'BsmtFinType1': 'NA', 'BsmtFinType2': 'NA', 'Electrical': train['Electrical'].mode()[0],
    'FireplaceQu': 'NA', 'GarageType': 'NA', 'GarageFinish': 'NA', 'GarageQual': 'NA',
    'GarageCond': 'NA', 'PoolQC': 'NA', 'Fence': 'NA', 'MiscFeature': 'None'
}

train.fillna(cat_fill_values, inplace=True)
test.fillna(cat_fill_values, inplace=True)

# Fill missing values for numerical features
train['LotFrontage'] = train.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
train.fillna({'MasVnrArea': 0, 'GarageYrBlt': train['YearBuilt'], 'BsmtFinSF1': 0, 'BsmtFinSF2': 0,
              'BsmtUnfSF': 0, 'TotalBsmtSF': 0, 'BsmtFullBath': 0, 'BsmtHalfBath': 0,
              'GarageCars': 0, 'GarageArea': 0}, inplace=True)

test['LotFrontage'] = test.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
test.fillna({'MasVnrArea': 0, 'GarageYrBlt': test['YearBuilt'], 'BsmtFinSF1': 0, 'BsmtFinSF2': 0,
             'BsmtUnfSF': 0, 'TotalBsmtSF': 0, 'BsmtFullBath': 0, 'BsmtHalfBath': 0,
             'GarageCars': 0, 'GarageArea': 0}, inplace=True)

# Feature Engineering
new_features = {
    'TotalSF': lambda df: df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF'],
    'Age': lambda df: df['YrSold'] - df['YearBuilt'],
    'Remodeled': lambda df: (df['YearRemodAdd'] != df['YearBuilt']).astype(int),
    'TotalBath': lambda df: df['BsmtFullBath'] + 0.5 * df['BsmtHalfBath'] + df['FullBath'] + 0.5 * df['HalfBath'],
    'OverallQual_GrLivArea': lambda df: df['OverallQual'] * df['GrLivArea'],
    'IsNew': lambda df: (df['YearBuilt'] > 2000).astype(int),
    'HighQuality': lambda df: (df['OverallQual'] >= 7).astype(int),
    'YearsSinceRemodel': lambda df: df['YrSold'] - df['YearRemodAdd'],
    'SaleMonth': lambda df: df['MoSold'],
    'HasPool': lambda df: (df['PoolArea'] > 0).astype(int),
    'HasFireplace': lambda df: (df['Fireplaces'] > 0).astype(int),
    'HasGarage': lambda df: (df['GarageArea'] > 0).astype(int),
    'OverallScore': lambda df: df['OverallQual'] + df['OverallCond'],
    'BasementScore': lambda df: df['BsmtQual'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}) + df['BsmtCond'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}),
    'GarageScore': lambda df: df['GarageQual'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}) + df['GarageCond'].replace({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0})
}

for feature, func in new_features.items():
    train[feature] = func(train)
    test[feature] = func(test)

# Separate the target variable
train_target = train['SalePrice']
train_features = train.drop(['SalePrice'], axis=1)

# Encode features for modeling
cat_features = ['MSZoning', 'Neighborhood', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
                'Foundation', 'BsmtQual', 'BsmtExposure', 'HeatingQC', 'KitchenQual', 'GarageType',
                'GarageFinish', 'SaleType', 'SaleCondition']

encoder = TargetEncoder(cols=cat_features)
train_encoded = encoder.fit_transform(train_features, train_target)
test_encoded = encoder.transform(test)

# Ensure 'SalePrice' is in train_encoded after alignment
train_encoded['SalePrice'] = train_target

# Align the train and test datasets to have the same columns
train_encoded, test_encoded = train_encoded.align(test_encoded, join='inner', axis=1)

# Add back 'SalePrice' if it was dropped during alignment
if 'SalePrice' not in train_encoded.columns:
    train_encoded['SalePrice'] = train_target

# Verify if all features are numerical
train_encoded = pd.get_dummies(train_encoded, drop_first=True)
test_encoded = pd.get_dummies(test_encoded, drop_first=True)

# Align the datasets again to ensure same columns
train_encoded, test_encoded = train_encoded.align(test_encoded, join='inner', axis=1)
train_encoded['SalePrice'] = train_target

# Model-based feature importance
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(train_encoded.drop('SalePrice', axis=1), train_encoded['SalePrice'])

feature_importances = pd.Series(model.feature_importances_, index=train_encoded.drop('SalePrice', axis=1).columns)
top_features = feature_importances.nlargest(20).index.tolist()
print(f'Top Features: {top_features}')

# Recursive Feature Elimination (RFE)
rfe = RFE(estimator=model, n_features_to_select=20)
rfe.fit(train_encoded.drop('SalePrice', axis=1), train_encoded['SalePrice'])

rfe_features = train_encoded.drop('SalePrice', axis=1).columns[rfe.support_].tolist()
print(f'RFE Selected Features: {rfe_features}')

# Combine features from model importance and RFE
combined_features = list(set(top_features).union(set(rfe_features)))

# Use these features for model training
X_train = train_encoded[combined_features]
y_train = train_encoded['SalePrice']
X_test = test_encoded[combined_features]

# Train the model
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

xg_reg = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xg_reg.fit(X_train, y_train)

y_pred_xg = xg_reg.predict(X_val)
rmse_xg = np.sqrt(mean_squared_error(y_val, y_pred_xg))
print(f'RMSE (XGBoost): {rmse_xg}')

# Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_val)
rmse_lin = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'RMSE (Linear Regression): {rmse_lin}')

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))
print(f'RMSE (Random Forest): {rmse_rf}')

# Hyperparameter Tuning for Gradient Boosting using Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3]
}

grid_search = GridSearchCV(estimator=XGBRegressor(objective='reg:squarederror'), param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_val)
rmse_best = np.sqrt(mean_squared_error(y_val, y_pred_best))
print(f'Best RMSE: {rmse_best}')

# Predict on the test dataset using the best model
final_predictions = best_model.predict(X_test)

# Prepare the submission file
submission = pd.DataFrame({
    'Id': test_id,
    'SalePrice': final_predictions
})

submission.to_csv('submission.csv', index=False)